In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import re
import requests
import collections
import functools
import itertools

In [120]:
headers = {
    "Host": "adventofcode.com",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "DNT": "1",
    "Referer": "http://adventofcode.com/2016/day/11",
    "Accept-Encoding": "gzip, deflate, sdch",
    "Accept-Language": "en-US,en;q=0.8,ru;q=0.6",
    "Cookie": "session=53616c7465645f5f5ebac3997c6c17b36bb86dd7a54e710a5121d7aba9fb1bd44020840204e963daf3388cece92610a2",
}

r = requests.get('http://adventofcode.com/2016/day/23/input', headers=headers)
input = r.text.strip()
lines = input.split('\n')
lines

['cpy a b',
 'dec b',
 'cpy a d',
 'cpy 0 a',
 'cpy b c',
 'inc a',
 'dec c',
 'jnz c -2',
 'dec d',
 'jnz d -5',
 'dec b',
 'cpy b c',
 'cpy c d',
 'dec d',
 'inc c',
 'jnz d -2',
 'tgl c',
 'cpy -16 c',
 'jnz 1 c',
 'cpy 75 c',
 'jnz 85 d',
 'inc a',
 'inc d',
 'jnz d -2',
 'inc c',
 'jnz c -5']

In [165]:
input = """cpy a b
dec b
cpy a d
cpy 0 a
 cpy b c
 mul d c
 add c a
 nop
 nop
 nop
dec b
cpy b c
cpy c d
dec d
inc c
jnz d -2
tgl c
cpy -16 c
jnz 1 c
cpy 75 c
jnz 85 d
inc a
inc d
jnz d -2
inc c
jnz c -5"""
lines = input.split('\n')

In [104]:
class VAR:
    def __init__(self, a):
        try:
            int(a)
            self.num = True
            self.value = int(a)
        except Exception:
            self.num = False
            self.label = a
    
    def get(self):
        if self.num:
            return self.value
        if self.label not in VARS:
            VARS[self.label] = 0
        return VARS[self.label]
    
    def __repr__(self):
        return f'N {self.value}' if self.num else f'V {self.label} {self.get()}'

In [180]:
class CPY:
    def act(self, a, b, p):
        if b.num:
            return p + 1
        VARS[b.label] = a.get()
        return p + 1
    
    def tgl(self):
        return JNZ()
    
    def __repr__(self):
        return 'CPY'
        
class INC:
    def act(self, a, b, p):
        if a.num:
            return p + 1
        VARS[a.label] = a.get() + 1
        return p + 1
    
    def tgl(self):
        return DEC()
    
    def __repr__(self):
        return 'INC'
    
class DEC:
    def act(self, a, b, p):
        if a.num:
            return p + 1
        VARS[a.label] = a.get() - 1
        return p + 1
    
    def tgl(self):
        return INC()
    
    def __repr__(self):
        return 'DEC'
    
class JNZ:
    def act(self, a, b, p):
        if a.get() == 0:
            return p + 1
        return p + b.get()
    
    def tgl(self):
        return CPY()
    
    def __repr__(self):
        return 'JNZ'
    
class TGL:
    def act(self, a, b, p):
        x = a.get() + p
        if x >= 0 and x < len(ACTIONS):
            ACTIONS[x] = ACTIONS[x].tgl()
        return p + 1
    
    def tgl(self):
        return INC()
    
    def __repr__(self):
        return 'TGL'
    
class ADD:
    def act(self, a, b, p):
        VARS[b.label] = a.get() + b.get()
        return p + 1
    
    def tgl(self):
        print('TOGGLED ADD')
    
    def __repr__(self):
        return 'ADD'
    
class MUL:
    def act(self, a, b, p):
        VARS[b.label] = b.get() * a.get()
        return p + 1
    
    def tgl(self):
        print('TOGGLED MUL')
    
    def __repr__(self):
        return 'MUL'
    
class NOP:
    def act(self, a, b, p):
        return p + 1
    
    def tgl(self):
        print('TOGGLED NOP')
    
    def __repr__(self):
        return 'NOP'
    
class OUT:
    def act(self, a, b, p):
        OUTs.append(VARS[a.label])
        return p + 1
    
    def tgl(self):
        print('TOGGLED OUT')
    
    def __repr__(self):
        return 'OUT'

In [166]:
ARGS = []
ACTIONS = []

for line in lines:
    for a, b in re.findall(r'cpy (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(CPY())
    for a in re.findall(r'inc (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(INC())
    for a in re.findall(r'dec (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(DEC())
    for a, b in re.findall(r'jnz (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(JNZ())
    for a in re.findall(r'tgl (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(TGL())
    for a, b in re.findall(r'add (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(ADD())
    for a, b in re.findall(r'mul (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(MUL())
    for i in re.findall(r'nop', line):
        ARGS.append([None, None])
        ACTIONS.append(NOP())
    
len(lines), len(ACTIONS), len(ARGS)

(26, 26, 26)

In [140]:
p = 0
c = 0
VARS = {}    
VARS['a'] = 7
while p < len(ACTIONS):
#     print(p, VARS)
#     if p > 5:
#         break
#     print(ACTIONS)
    p = ACTIONS[p].act(ARGS[p][0], ARGS[p][1], p)
    c += 1
    if c % 1000000 == 0:
        print(p, VARS)

In [133]:
VARS

{'a': 11415, 'b': 1, 'c': 0, 'd': 0}

In [141]:
VARS

{'a': 11415, 'b': 1, 'c': 0, 'd': 0}

In [167]:
p = 0
c = 0
VARS = {}    
VARS['a'] = 12
while p < len(ACTIONS):
#     print(p, VARS)
#     if p > 5:
#         break
#     print(ACTIONS)
    p = ACTIONS[p].act(ARGS[p][0], ARGS[p][1], p)
    c += 1
    if c % 10000000 == 0:
        print(p, VARS)

In [169]:
VARS

{'a': 479007975, 'b': 1, 'c': 0, 'd': 0}

In [174]:
input = """cpy a d
cpy 15 c
cpy 170 b
inc d
dec b
jnz b -2
dec c
jnz c -5
cpy d a
jnz 0 0
cpy a b
cpy 0 a
cpy 2 c
jnz b 2
jnz 1 6
dec b
dec c
jnz c -4
inc a
jnz 1 -7
cpy 2 b
jnz c 2
jnz 1 4
dec b
dec c
jnz 1 -4
jnz 0 0
out b
jnz a -19
jnz 1 -21"""
lines = input.split('\n')

In [181]:
ARGS = []
ACTIONS = []

for line in lines:
    for a, b in re.findall(r'cpy (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(CPY())
    for a in re.findall(r'inc (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(INC())
    for a in re.findall(r'dec (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(DEC())
    for a, b in re.findall(r'jnz (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(JNZ())
    for a in re.findall(r'tgl (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(TGL())
    for a in re.findall(r'out (.+)', line):
        a = VAR(a)
        ARGS.append([a, None])
        ACTIONS.append(OUT())
    for a, b in re.findall(r'add (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(ADD())
    for a, b in re.findall(r'mul (.+) (.+)', line):
        a = VAR(a)
        b = VAR(b)
        ARGS.append([a, b])
        ACTIONS.append(MUL())
    for i in re.findall(r'nop', line):
        ARGS.append([None, None])
        ACTIONS.append(NOP())
    
len(lines), len(ACTIONS), len(ARGS)

(30, 30, 30)

In [201]:
def gen(a):
    p = 0
    c = 0
    global OUTs, VARS
    VARS = {}    
    VARS['a'] = a
    OUTs = []
    while p < len(ACTIONS) and len(OUTs) < 20:
        p = ACTIONS[p].act(ARGS[p][0], ARGS[p][1], p)
        c += 1
        if c % 1000000 == 0:
            print(p, VARS)
    return OUTs

In [209]:
for i in range(1000):
    a = gen(i)
    print(i, a)
    if a == ([0, 1] * 10):
        break

0 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1]
1 [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]
2 [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
3 [1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]
4 [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]
5 [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
6 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
7 [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
8 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
9 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
11 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
12 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
13 [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
14 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
15 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0

130 [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0]
131 [1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0]
132 [0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0]
133 [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0]
134 [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0]
135 [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0]
136 [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0]
137 [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
138 [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
139 [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
140 [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
141 [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1]
142 [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]
143 [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
144 [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1]
145 [1, 1, 1, 0, 0, 0, 0,